In [24]:
import torch
from torch import nn
import torch.nn.functional as F
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
words = open("names.txt").read().splitlines()
random.seed(42)
random.shuffle(words)

In [26]:
chars = ['.'] + sorted(list(set(''.join(words))))
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for s, i in stoi.items()}

In [27]:
nsize = len(words)
tsize = int(0.8*nsize)
trainset = words[:tsize]
testset = words[tsize:]

## Count Based

In [28]:
N = torch.zeros((27, 27, 27), dtype=torch.int)

In [29]:
# calculating counts of trigrams
for w in trainset[:]:
    w = ['.'] + list(w) + ['.']
    for i in range(len(w)-2):
        a, b, c = w[i], w[i+1], w[i+2]
        N[stoi[a], stoi[b], stoi[c]] += 1

In [30]:
# probabilites from counts
P = (N+1).float()
P /= P.sum(2,keepdim=True)

In [31]:
# bigram probabilites, helpful in generating the first two tokens for sampling
B = N.sum(2, keepdim=True).float()
B = B/B.sum(1, keepdim=True)
B = B.reshape((27,27))

In [32]:
# generating from the trigram model
for i in range(10):
    ix1 = 0
    ix2 = torch.multinomial(B[ix1], num_samples=1, replacement=True).item()
    outputs = [itos[ix2]]
    while True:
        p = P[ix1, ix2]
        x = torch.multinomial(p, num_samples=1, replacement=True).item()
        outputs.append(itos[x])
        ix1 = ix2
        ix2 = x
        if x == 0:
            break
    print(''.join(outputs))

dasid.
ardene.
luwaynn.
lanid.
cansona.
layaria.
formarmiah.
xfj.
cassaj.
omne.


In [33]:
# evaluating loss on trainset
nll = 0.0
cnt = 0.0
for w in trainset[:]:
    w = ['.'] + list(w) + ['.']
    for i in range(len(w)-2):
        a, b, c = w[i], w[i+1], w[i+2]
        prob = P[stoi[a], stoi[b], stoi[c]]
        cnt+=1
        logprob = torch.log(prob)
        nll += logprob
        # print(f'{a} {b} {c} : {prob.item():.4f} {logprob.item():.4f}')
nll = -nll/cnt
print(nll.item())

2.0966954231262207


In [34]:
# evaluating loss on testset
nll = 0.0
cnt = 0.0
for w in testset[:]:
    w = ['.'] + list(w) + ['.']
    for i in range(len(w)-2):
        a, b, c = w[i], w[i+1], w[i+2]
        prob = P[stoi[a], stoi[b], stoi[c]]
        cnt+=1
        logprob = torch.log(prob)
        nll += logprob
        # print(f'{a} {b} {c} : {prob.item():.4f} {logprob.item():.4f}')
nll = -nll/cnt
print(nll.item())

2.12101674079895


## Neural Network

In [35]:
xs = []
ys = []
xst = []
yst = []

for w in trainset[:]:
    w = ['.'] + list(w) + ['.']
    for i in range(len(w)-2):
        a, b, c = w[i], w[i+1], w[i+2]
        xs.append([stoi[a], stoi[b]])
        ys.append(stoi[c])

for w in testset[:]:
    w = ['.'] + list(w) + ['.']
    for i in range(len(w)-2):
        a, b, c = w[i], w[i+1], w[i+2]
        xst.append([stoi[a], stoi[b]])
        yst.append(stoi[c])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
xst = torch.tensor(xst)
yst = torch.tensor(yst)

In [36]:
# stacking one hot encoded vectors train
a1 = F.one_hot(xs[:,0], num_classes=27)
a2 = F.one_hot(xs[:,1], num_classes=27)
xenc = torch.hstack((a1,a2)).float()
nums = xenc.shape[0]

# stacking one hot encoded vectors test 
b1 = F.one_hot(xst[:,0], num_classes=27)
b2 = F.one_hot(xst[:,1], num_classes=27)
xenct = torch.hstack((b1,b2)).float()
nums = xenc.shape[0]

In [37]:
# one layer model
model = nn.Linear(54, 27)

In [38]:
# lossfn for calculating loss and optimizer for upadting gradient
lossfn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [39]:
# train
epochs = 500
epfact = epochs/10
for i in range(epochs):
    # train
    y = model(xenc)
    
    loss = lossfn(y, ys)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # test
    yt = model(xenct)
    losst = lossfn(yt, yst)
    
    if i % epfact == 0:
        print(f'train: {loss.item():.4f} || test: {losst.item():.4f}')

train: 3.3325 || test: 3.1860
train: 2.2534 || test: 2.2545
train: 2.2409 || test: 2.2429
train: 2.2386 || test: 2.2411
train: 2.2378 || test: 2.2405
train: 2.2373 || test: 2.2403
train: 2.2371 || test: 2.2402
train: 2.2369 || test: 2.2402
train: 2.2368 || test: 2.2402
train: 2.2368 || test: 2.2402


In [41]:
# generate
for i in range(10):
    ix1 = 0
    ix2 = torch.multinomial(B[ix1], num_samples=1, replacement=True).item()
    outputs = [itos[ix2]]
    while True:
        x1 = F.one_hot(torch.tensor(ix1), num_classes=27)
        x2 = F.one_hot(torch.tensor(ix2), num_classes=27)
        xen = torch.hstack((x1,x2)).float()
        y = model(xen)
        p = torch.softmax(y,0)
        x = torch.multinomial(p, num_samples=1, replacement=True).item()
        outputs.append(itos[x])
        ix1 = ix2
        ix2 = x
        if x == 0:
            break
    print(''.join(outputs))

ka.
amiven.
merona.
erh.
jechanyon.
lynnntaleydonjamirestaeriosepon.
dish.
beylayah.
peakhryanzyph.
mber.
